# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [217]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [218]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [219]:
data

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
...,...,...,...,...,...,...,...
68579,94547,Gerald,Donovan,687884,211,Wine - Prosecco Valdobiaddene,24
68580,57654,Lashonda,Roy,4373415,211,Wine - Prosecco Valdobiaddene,15
68581,36751,Glen,Frederick,2405597,211,Wine - Prosecco Valdobiaddene,10
68582,43270,Devon,Mc Daniel,4328852,211,Wine - Prosecco Valdobiaddene,11


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [220]:
data1 = data.groupby(['CustomerID','ProductName'], as_index = True).sum()
data1

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1
...                                           ...        ...       ...
98200      Vol Au Vents                   5160346        272        50
           Wasabi Powder                  4438427         80        25
           Wine - Fume Blanc Fetzer       6226060         76        25
           Wine - Hardys Bankside Shiraz   266676        360        25
           Yogurt - French Vanilla        2112061        320        25

[63628 rows x 3 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [221]:
pivot = pd.pivot_table(data1, values='Quantity', columns='CustomerID', index='ProductName',
                      fill_value=0)
pivot

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [222]:
pivot2= pivot.transpose()
pdist(pivot2, metric='euclidean')
squareform(pdist(pivot2, metric='euclidean'))
(1 + squareform(pdist(pivot2, metric='euclidean')))
dist_eucl = 1 / (1 + squareform(pdist(pivot2, metric='euclidean')))
dist_eucl
distances = pd.DataFrame(dist_eucl, index=pivot2.index, 
                         columns=pivot2.index)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [223]:
similarities = list(distances[200].sort_values(ascending=False)[1:6].index)
similarities

[3317, 1072, 3535, 1920, 3909]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [224]:
data2 = data1.groupby(['CustomerID','ProductName'], as_index = True).sum()
data2

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1
...                                           ...        ...       ...
98200      Vol Au Vents                   5160346        272        50
           Wasabi Powder                  4438427         80        25
           Wine - Fume Blanc Fetzer       6226060         76        25
           Wine - Hardys Bankside Shiraz   266676        360        25
           Yogurt - French Vanilla        2112061        320        25

[63628 rows x 3 columns]

In [225]:
#Esta pivot table regresa un output erroneo

pivot_wrong = pd.DataFrame(pivot, columns=similarities)
pivot_wrong = pivot.T
pivot_wrong

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0,0,0,25,0,50,0,25,0,0,...,0,25,25,0,0,0,0,0,0,0
98069,0,0,0,25,0,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0
98159,0,0,0,0,0,0,0,0,0,0,...,0,50,0,0,0,0,25,0,0,0


In [226]:
data3 = data2.loc[similarities]
data3

SalesID  ProductID  Quantity
CustomerID ProductName                                                 
3317       Bay Leaf                        2240383        202         1
           Beef - Ground Medium            6007552         45         1
           Beef - Rib Eye Aaa              2367688        244         1
           Butter - Unsalted               4278889        402         1
           Cheese - Victor Et Berthold      703028        316         1
...                                            ...        ...       ...
3909       Wine - Charddonnay Errazuriz    9045484        450         2
           Wine - Fume Blanc Fetzer        3349679         76         1
           Wine - Magnotta, Merlot Sr Vqa  3566710        192         1
           Wine - Toasted Head             4134000        134         1
           Wine - Valpolicella Masi        3584400        296         1

[262 rows x 3 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [227]:
quantity_rank = data3.reset_index().drop(columns=['CustomerID','SalesID','ProductID']).groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
quantity_rank

,Quantity
ProductName,
Soup - Campbells Bean Medley,4
Muffin - Carrot Individual Wrap,3
Bay Leaf,3
Pork - Kidney,3
"Pepper - Black, Whole",3
...,...
Kiwi,1
Knife Plastic - White,1
Lambcasing,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [228]:
data_merge = pd.merge(left=quantity_rank, right=pivot, on='ProductName')
data_merge = data_merge[data_merge[200] == 0].sort_values(by='Quantity', ascending=False)[1:6]
data_merge = list(data_merge.index)
data_merge

['Bay Leaf',
 'Pork - Kidney',
 'Wanton Wrap',
 'Muffin - Carrot Individual Wrap',
 'Wine - Blue Nun Qualitatswein']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [229]:
customer_recommend = {}
# customer_list = [2503,  264, 3317, 3305, 1072, 3535]  
customer_list = list(data.CustomerID.unique())
customer_list.sort()

for customer in customer_list:
    similarities = list(distances[customer].sort_values(ascending=False)[1:6].index)
    data1 = data.groupby(['CustomerID', 'ProductName'], as_index=False).agg({'Quantity':'sum'})
    data2 = data1[data1.CustomerID.isin(similarities)].sort_values(by='Quantity', ascending=False)
    data3 = data2.groupby(['ProductName'], as_index=True).agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending=False)
    data_merge = pd.merge(left=data3, right=pivot, on='ProductName')
    data_merge = list(data_merge[data_merge[customer] == 0].sort_values(by='Quantity', ascending=False)[1:6].index)
    
    customer_recommend[customer] = data_merge

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [230]:
recommendations_per_customer = pd.DataFrame.from_dict(customer_recommend)
recommendations_per_customer

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Soup - Campbells Bean Medley,Bay Leaf,Bread - Italian Roll With Herbs,"Veal - Inside, Choice",Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,Wine - Ej Gallo Sierra Valley,Appetizer - Mushroom Tart,Wine - Ej Gallo Sierra Valley,...,"Lentils - Red, Dry",Papayas,Potatoes - Idaho 100 Count,Ice Cream Bar - Oreo Cone,Rum - Mount Gay Eclipes,"Cheese - Brie,danish",Cheese - Taleggio D.o.p.,"Wine - Red, Harrow Estates, Cab",Pernod,Sherry - Dry
1,Wine - Blue Nun Qualitatswein,Pork - Kidney,"Veal - Inside, Choice","Beets - Candy Cane, Organic","Pepper - Black, Whole","Peas - Pigeon, Dry","Veal - Inside, Choice",Yoghurt Tubes,"Peas - Pigeon, Dry","Pepper - Black, Whole",...,Meldea Green Tea Liquor,Meldea Green Tea Liquor,Pants Custom Dry Clean,Halibut - Fletches,Tofu - Firm,Wiberg Super Cure,Beans - Kidney White,Bread - Raisin Walnut Oval,Tea - Jasmin Green,Pork - Hock And Feet Attached
2,Wine - Ej Gallo Sierra Valley,Wanton Wrap,"Fish - Scallops, Cold Smoked","Nut - Chestnuts, Whole",Soup - Campbells Bean Medley,Bar - Granola Trail Mix Fruit Nut,"Pepper - Black, Whole","Pepper - Black, Whole","Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,...,Muffin - Carrot Individual Wrap,Wine - Chablis 2003 Champs,Cake - Box Window 10x10x2.5,"Wine - Red, Colio Cabernet",Sobe - Tropical Energy,Pants Custom Dry Clean,Milk - 1%,Tea - English Breakfast,Pastry - Choclate Baked,Pears - Bosc
3,Beans - Wax,Muffin - Carrot Individual Wrap,Fondant - Icing,Wasabi Powder,"Sauce - Gravy, Au Jus, Mix","Veal - Inside, Choice",Cheese - Cambozola,Soupfoamcont12oz 112con,Lamb - Ground,Sardines,...,Coffee - Dark Roast,Mussels - Frozen,Cod - Black Whole Fillet,Grenadine,"Cheese - Boursin, Garlic / Herbs",Shrimp - 31/40,Yoghurt Tubes,Wine - Redchard Merritt,"Peas - Pigeon, Dry","Soup - Campbells, Cream Of"
4,Scallops 60/80 Iqf,Wine - Blue Nun Qualitatswein,Wine - Two Oceans Cabernet,"Cheese - Boursin, Garlic / Herbs",Cheese - Cambozola,Sausage - Breakfast,Sprouts - Baby Pea Tendrils,"Wine - Magnotta, Merlot Sr Vqa",Butter - Unsalted,Butter - Unsalted,...,Fuji Apples,French Pastry - Mini Chocolate,Rum - Mount Gay Eclipes,Garlic - Peeled,"Rosemary - Primerba, Paste","Placemat - Scallop, White",Sprouts - Baby Pea Tendrils,Butter - Unsalted,Lettuce - Spring Mix,Ecolab - Mikroklene 4/4 L


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [233]:
# Para método coseno

metrica = 'cosine'

# Obtener lista matriz de distancias
print(f'Obteniendo matriz de distancias de la métrica: {0}'.format(metrica))
pivot = pd.pivot_table(data1, values='Quantity', columns='CustomerID', index='ProductName', fill_value=0)
pivot2= pivot.transpose()
dist_eucl = 1 / (1 + squareform(pdist(pivot2, metric='cosine')))
distances = pd.DataFrame(dist_eucl, index=pivot2.index, columns=pivot2.index)
print(f'Matriz de distancias de la métrica {0} obtenida'.format(metrica))


customer_recommend = {}
customer_list = list(data.CustomerID.unique())
customer_list.sort()

print(f'Iterando sobre lista de clientes')

for customer in customer_list:
    similarities = list(distances[customer].sort_values(ascending=False)[1:6].index)
    data1 = data.groupby(['CustomerID', 'ProductName'], as_index=False).agg({'Quantity':'sum'})
    data2 = data1[data1.CustomerID.isin(similarities)].sort_values(by='Quantity', ascending=False)
    data3 = data2.groupby(['ProductName'], as_index=True).agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending=False)
    data_merge = pd.merge(left=data3, right=pivot, on='ProductName')
    data_merge = list(data_merge[data_merge[customer] == 0].sort_values(by='Quantity', ascending=False)[1:6].index)
    
    customer_recommend[customer] = data_merge

print(f'Generando tabla de recomendaciones')
recommendations_per_customer = pd.DataFrame.from_dict(customer_recommend)
recommendations_per_customer


Obteniendo matriz de distancias de la métrica: 0
Matriz de distancias de la métrica 0 obtenida
Iterando sobre lista de clientes
Generando tabla de recomendaciones


,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,"Soup - Campbells, Beef Barley",Snapple Lemon Tea,"Water - Mineral, Natural",Olive - Spread Tapenade,Durian Fruit,Bread - Bistro White,Olive - Spread Tapenade,Crab - Imitation Flakes,Napkin White - Starched,Wine - Blue Nun Qualitatswein,...,Cheese - Cottage Cheese,Blackberries,French Pastry - Mini Chocolate,Bananas,Kellogs Special K Cereal,Beef - Montreal Smoked Brisket,Muffin Batt - Blueberry Passion,Bananas,Cheese - Taleggio D.o.p.,Cheese - Cottage Cheese
1,Onions - Cippolini,General Purpose Trigger,Snapple - Iced Tea Peach,Bagel - Plain,Wine - Hardys Bankside Shiraz,Sausage - Breakfast,Spinach - Baby,Cheese - Parmesan Cubes,"Mushroom - Porcini, Dry",Pickerel - Fillets,...,Tomatoes Tear Drop,Coffee - Dark Roast,Scallop - St. Jaques,Initation Crab Meat,Pears - Bosc,Blackberries,Juice - Orange,Wine - Redchard Merritt,Squid U5 - Thailand,Wine - Toasted Head
2,Tea - Herbal Sweet Dreams,Tomatoes Tear Drop,Wine - Ej Gallo Sierra Valley,Pork - Hock And Feet Attached,Bread - Raisin Walnut Oval,"Juice - Cranberry, 341 Ml",Veal - Osso Bucco,Whmis - Spray Bottle Trigger,Wine - Pinot Noir Latour,Beer - Rickards Red,...,Ice Cream Bar - Hageen Daz To,Hersey Shakes,Wine - Pinot Noir Latour,"Pork - Loin, Center Cut",Langers - Ruby Red Grapfruit,Cheese Cloth No 100,Beer - Original Organic Lager,Chocolate - Compound Coating,"Cheese - Brie, Triple Creme","Shrimp - Baby, Warm Water"
3,Muffin - Zero Transfat,"Thyme - Lemon, Fresh",French Pastry - Mini Chocolate,Wiberg Super Cure,Gatorade - Xfactor Berry,Garbage Bags - Clear,Carbonated Water - Blackcherry,Bouq All Italian - Primerba,Pork - Inside,"Hickory Smoke, Liquid",...,Table Cloth 120 Round White,Papayas,"Beef - Tenderlion, Center Cut","Sole - Dover, Whole, Fresh","Sauce - Gravy, Au Jus, Mix",Cake - Mini Cheesecake,"Nut - Chestnuts, Whole",Longos - Grilled Salmon With Bbq,Crackers - Trio,Island Oasis - Mango Daiquiri
4,Banana Turning,Veal - Osso Bucco,Tray - 16in Rnd Blk,Oil - Shortening - All - Purpose,Sea Bass - Whole,Beef - Montreal Smoked Brisket,Dc - Frozen Momji,Apricots Fresh,Chinese Foods - Chicken,Mussels - Frozen,...,Bananas,Carbonated Water - Cherry,Water - Green Tea Refresher,Bread - Italian Roll With Herbs,Beef - Rib Eye Aaa,"Soup - Campbells, Lentil",Flavouring - Orange,Cumin - Whole,Pernod,Jolt Cola - Electric Blue
